# Data ETL Pipeline - Interactive Example 📊

This notebook demonstrates a Data ETL Pipeline for loading, cleaning, and analyzing sales data.

**Key steps:**
- Load and validate raw data
- Clean and transform the data
- Generate aggregated reports
- Visualize key insights

## 🚀 Getting Started

To run this notebook, launch Jupyter Lab with the required dependencies:

```bash
uvx --with "flowerpower[rq],pandas>=2.0.0,matplotlib,seaborn" jupyter lab
```

## 📦 Setup & Dependencies

Import necessary libraries and set up the environment.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Add FlowerPower src to path
sys.path.insert(0, str(Path.cwd().parents[2] / "src"))

from flowerpower.pipeline.manager import PipelineManager

print("✅ Dependencies loaded and environment set up.")

## 🔧 Initialize FlowerPower Pipeline

Set up the pipeline manager to execute our ETL workflow.

In [ ]:
pipeline_manager = PipelineManager(base_dir=".", fs=None)
print(f"✅ Pipeline manager initialized. Available pipelines: {pipeline_manager.list_pipelines()}")

## 📊 Explore Sample Data

Examine the raw sales data before processing.

In [ ]:
sales_data = pd.read_csv("data/sales_data.csv")
print(f"Dataset shape: {sales_data.shape}")
print(f"Missing values: {sales_data.isnull().sum().sum()}")
print(f"Duplicate rows: {sales_data.duplicated().sum()}")
sales_data.head()

## 🎯 Run ETL Pipeline

Execute the `sales_etl` pipeline to process the data.

In [ ]:
result = pipeline_manager.run(
    "sales_etl",
    final_vars=["clean_data", "sales_summary", "validation_report"]
)

clean_data = result['clean_data']
sales_summary = result['sales_summary']
validation_report = result['validation_report']

print("✅ Pipeline execution completed!")

### 🔍 Validation Report

Check the data validation results.

In [ ]:
print(f"Data is valid: {validation_report['is_valid']}")
print(f"Price violations: {validation_report['price_violations']}")
if validation_report['missing_values']:
    print(f"Missing values: {validation_report['missing_values']}")

### 🧹 Clean Data

Inspect the cleaned data.

In [ ]:
print(f"Clean data shape: {clean_data.shape}")
print(f"Missing values after cleaning: {clean_data.isnull().sum().sum()}")
print(f"Total revenue: ${clean_data['total_sales'].sum():,.2f}")
clean_data.head()

### 📈 Sales Summary

Review the aggregated sales summary.

In [ ]:
print(f"Sales summary shape: {sales_summary.shape}")
sales_summary.head(10)

## 📊 Data Visualization

Visualize the processed data to gain insights.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Sales Data Analysis', fontsize=16)

# Revenue by Product
product_revenue = sales_summary.groupby('product')['total_sales'].sum().sort_values(ascending=False)
axes[0, 0].bar(product_revenue.index, product_revenue.values)
axes[0, 0].set_title('Revenue by Product')
axes[0, 0].tick_params(axis='x', rotation=45)

# Sale Amount Distribution
axes[0, 1].hist(clean_data['total_sales'], bins=30, alpha=0.7)
axes[0, 1].set_title('Sale Amount Distribution')

# Customer Spending
customer_spending = sales_summary.groupby('customer')['total_sales'].sum().sort_values(ascending=False)
axes[1, 0].bar(customer_spending.index, customer_spending.values)
axes[1, 0].set_title('Customer Total Spending')

# Quantity vs Revenue
axes[1, 1].scatter(sales_summary['total_quantity'], sales_summary['total_sales'], alpha=0.6)
axes[1, 1].set_title('Quantity vs Revenue')

plt.tight_layout()
plt.show()

## ⚙️ Pipeline Configuration Experiments

Test how different configurations affect the output. Here, we'll test a stricter price validation.

In [ ]:
strict_result = pipeline_manager.run(
    "sales_etl",
    inputs={"min_price": 5.0, "max_price": 500.0},
    final_vars=["validation_report"]
)
strict_validation = strict_result['validation_report']

print(f"Original price violations: {validation_report['price_violations']}")
print(f"Strict price violations: {strict_validation['price_violations']}")

## 💾 Export and Save Results

Save the cleaned data, sales summary, and validation report.

In [ ]:
output_dir = Path("notebook_outputs")
output_dir.mkdir(exist_ok=True)

clean_data.to_csv(output_dir / "cleaned_sales_data.csv", index=False)
sales_summary.to_csv(output_dir / "sales_summary.csv", index=False)
with open(output_dir / "validation_report.json", "w") as f:
    json.dump(validation_report, f, indent=2, default=str)

print(f"✅ Outputs saved to: {output_dir.absolute()}")

## 🔗 Learn More

To run the script version of this pipeline:

```bash
uv run scripts/run_example.py sync
```